In [0]:
# Define schema

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)])

schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                            StructField("driverRef", StringType(), True),
                            StructField("number", IntegerType(), True),
                            StructField("code", StringType(), True),
                            StructField("name", name_schema),
                            StructField("dob", DateType(), True),
                            StructField("nationality", StringType(), True),
                            StructField("url", StringType(), True)
                            ])

In [0]:
# Read json file
drivers_df = spark.read.json("/mnt/formula1dataplatformdl/raw/drivers.json", schema = schema)
display(drivers_df)

In [0]:
# Process data

from pyspark.sql.functions import col, concat, current_timestamp, lit

drivers_processed = drivers_df.withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("driverRef", "driver_ref") \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
    .drop(col("url"))

display(drivers_processed)

In [0]:
# Write df to parquet
drivers_processed.write.parquet("/mnt/formula1dataplatformdl/processed/drivers", mode = "overwrite")

In [0]:
# Check if data has been properly written
display(spark.read.parquet("/mnt/formula1dataplatformdl/processed/drivers"))